In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv('/content/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
data.shape

(50000, 2)

In [4]:
data.value_counts()

,,count
review,sentiment,
"Loved today's show!!! It was a variety and not solely cooking (which would have been great too). Very stimulating and captivating, always keeping the viewer peeking around the corner to see what was coming up next. She is as down to earth and as personable as you get, like one of us which made the show all the more enjoyable. Special guests, who are friends as well made for a nice surprise too. Loved the 'first' theme and that the audience was invited to play along too. I must admit I was shocked to see her come in under her time limits on a few things, but she did it and by golly I'll be writing those recipes down. Saving time in the kitchen means more time with family. Those who haven't tuned in yet, find out what channel and the time, I assure you that you won't be disappointed.",positive,5
"Hilarious, clean, light-hearted, and quote-worthy. What else can you ask for in a film? This is my all-time, number one favorite movie. Ever since I was a little girl, I've dreamed of owning a blue van with flames and an observation bubble.<br /><br />The cliché characters in ridiculous situations are what make this film such great fun. The wonderful comedic chemistry between Stephen Furst (Harold) and Andy Tennant (Melio) make up most of my favorite parts of the movie. And who didn't love the hopeless awkwardness of Flynch? Don't forget the airport antics of Leon's cronies, dressed up as Hari Krishnas: dancing, chanting and playing the tambourine--unbeatable! The clues are genius, the locations are classic, and the plot is timeless.<br /><br />A word to the wise, if you didn't watch this film when you were little, it probably won't win a place in your heart today. But nevertheless give it a chance, you may find that ""It doesn't matter what you say, it doesn't matter what you do, you've gotta play.""",positive,4
"This show comes up with interesting locations as fast as the travel channel. It is billed as reality but in actuality it is pure prime time soap opera. It's tries to use exotic locales as a facade to bring people into a phony contest & then proceeds to hook viewers on the contestants soap opera style.<br /><br />It also borrows from an early CBS game show pioneer- Beat The Clock- by inventing situations for its contestants to try & overcome. Then it rewards the winner money. If they can spice it up with a little interaction between the characters, even better. While the game format is in slow motion versus Beat The Clock- the real accomplishment of this series is to escape reality. <br /><br />This show has elements of several types of successful past programs. Reality television, hardly, but if your hooked on the contestants, locale or contest, this is your cup of tea. If your not, this entire series is as I say, drivel dripping with gravy. It is another show hiding behind the reality label which is the trend it started in 2000.<br /><br />It is slick & well produced, so it might last a while yet. After all, so do re-runs of Gilligan's Island, Green Acres, The Beverly Hillbillies & The Brady Bunch. This just doesn't employ professional actors. The intelligence level is about the same.",negative,3
"When i got this movie free from my job, along with three other similar movies.. I watched then with very low expectations. Now this movie isn't bad per se. You get what you pay for. It is a tale of love, betrayal, lies, sex, scandal, everything you want in a movie. Definitely not a Hollywood blockbuster, but for cheap thrills it is not that bad. I would probably never watch this movie again. In a nutshell this is the kind of movie that you would see either very late at night on a local television station that is just wanting to take up some time, or you would see it on a Sunday afternoon on a local television station that is trying to take up some time. Despite the bad acting, cliché lines, and sub par camera work. I didn't have the desire to turn off the movie and pretend like it never popped into my DVD 

In [5]:
#positivee -> 1
#negative -> 0
data['sentiment']=data['sentiment'].replace({'positive':1,'negative':0})

In [6]:
data.head()
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=42)

In [9]:
train_data

,review,sentiment
39087,That's what I kept asking myself during the ma...,0
30893,I did not watch the entire movie. I could not ...,0
45278,A touching love story reminiscent of In the M...,1
16398,This latter-day Fulci schlocker is a totally a...,0
13653,"First of all, I firmly believe that Norwegian ...",0
...,...,...
11284,`Shadow Magic' recaptures the joy and amazemen...,1
44732,I found this movie to be quite enjoyable and f...,1
38158,Avoid this one! It is a terrible movie. So wha...,0
860,This production was quite a surprise for me. I...,1


In [10]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)


In [11]:
y_train=train_data['sentiment']
y_test=test_data['sentiment']

LSTM MODEL BUILDING


In [12]:
model=Sequential()
model.add(Embedding(input_dim=5000,output_dim=128,input_length=200))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [15]:
#create an instance of earlystopping
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [16]:
model.fit(X_train,y_train,epochs=5,batch_size=64,validation_split=0.2,callbacks=[earlystopping])

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 387ms/step - accuracy: 0.7246 - loss: 0.5310 - val_accuracy: 0.8593 - val_loss: 0.3419
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 197s 385ms/step - accuracy: 0.8602 - loss: 0.3431 - val_accuracy: 0.8618 - val_loss: 0.3577
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 384ms/step - accuracy: 0.8719 - loss: 0.3147 - val_accuracy: 0.8710 - val_loss: 0.3099
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 383ms/step - accuracy: 0.8873 - loss: 0.2770 - val_accuracy: 0.8475 - val_loss: 0.3655
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 196s 371ms/step - accuracy: 0.9044 - loss: 0.2458 - val_accuracy: 0.8619 - val_loss: 0.3249


In [17]:
model.save('model.h5')

In [20]:
import joblib
joblib.dump(tokenizer,'tokenizer.pkl')

['tokenizer.pkl']

In [18]:
loss,accuracy = model.evaluate(X_test,y_test)
print("Loss: ",loss,"Accuracy: ",accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 110ms/step - accuracy: 0.8735 - loss: 0.3044
Loss:  0.30163007974624634 Accuracy:  0.8769999742507935


In [21]:
#Building Predictive System
def predictive_system(review):
  review=tokenizer.texts_to_sequences([review])
  review=pad_sequences(review,maxlen=200)
  prediction=model.predict(review)
  return prediction[0][0]


In [48]:
result=predictive_system('The movie was a cinematic masterpiece')
if result > 0.45:
  print("Postive review ",result)
else:
  print("Negative review ",result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Postive review  0.84956497


In [41]:
print(tokenizer.word_index.get("good"))


49
